FIRST LET'S CREATE ALL OUR FOLDERS AND DO SOME HOUSEKEEPING

STRUCTURE 
    workspace
        -my_model_x
            -pretrained_model_x
            -model_data
                model
                data
            -images
    scripts (extra scripts)
    object_detection (tf object detection framework)

In [1]:
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
CUSTOM_MODEL_NAME = 'my_ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

ROOT_WORKSPACE = 'workspace_tiffanies'

In [2]:
import os
paths = {
    'APIMODEL_PATH': 'object_detection',
    'WORKSPACE_PATH': ROOT_WORKSPACE,
    'ANNOTATION_PATH': os.path.join(ROOT_WORKSPACE,CUSTOM_MODEL_NAME,'model_data','data'),
    'CHECKPOINT_PATH': os.path.join(ROOT_WORKSPACE,CUSTOM_MODEL_NAME,'model_data','model'),
    'OUTPUT_PATH': os.path.join(ROOT_WORKSPACE,CUSTOM_MODEL_NAME,'model_data','model', 'export'), 
    'TFJS_PATH':os.path.join(ROOT_WORKSPACE,CUSTOM_MODEL_NAME,'model_data','model','tfjsexport'), 
    'IMAGE_PATH': os.path.join(ROOT_WORKSPACE,'images'),
    'PRETRAINED_MODEL_PATH': os.path.join(ROOT_WORKSPACE,CUSTOM_MODEL_NAME),
    'SCRIPTS_PATH': 'scripts'
 }

In [3]:
#CREATE THESE FOLDERS
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [4]:
files = {
    'PIPELINE_CONFIG':os.path.join(paths['CHECKPOINT_PATH'], 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

NOW LET'S CREATE THE TF RECORDS AND THE LABELMAP FILE

In [5]:
#OUR CUSTOMLABELMAP
labels = [{'name':'burj', 'id':1},{'name':'empire', 'id':2}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tid:{}\n'.format(label['id']))
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('}\n')

In [6]:
#THE TF RECORDS
#CREATE TF RECORDS

#FIRST DOWNLOAD A SCRIPT FOR GENERATING TF RECORDS
#if not os.path.exists(files['TF_RECORD_SCRIPT']):
#    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}
        
        
#create the TF RECORDS        
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

Successfully created the TFRecord file: workspace_tiffanies\my_ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\model_data\data\train.record
Successfully created the TFRecord file: workspace_tiffanies\my_ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\model_data\data\test.record


CHECK IF TF RECORDS ARE OKAY

In [7]:
import tensorflow as tf
print(tf.__version__)

raw_dataset = tf.data.TFRecordDataset(os.path.join(paths['ANNOTATION_PATH'], 'test.record'))
for raw_record in raw_dataset.take(1):
    
    #print(raw_record.numpy())

    
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)
    
    #tf.io.parse_example(
    #example.SerializeToString(),
    #features = {'my_feature': tf.io.RaggedFeature(dtype=tf.int64)})


    #image = tf.decode_raw(features['image/encoded'], tf.uint8)
    #image = tf.decode_raw(features['image/encoded'], tf.uint8)
    #print(example)

2.7.0
features {
  feature {
    key: "image/encoded"
    value {
      bytes_list {
        value: "\377\330\377\341\036\321Exif\000\000MM\000*\000\000\000\010\000\021\001\000\000\003\000\000\000\001\017\300\000\000\001\001\000\003\000\000\000\001\013\320\000\000\001\002\000\003\000\000\000\003\000\000\000\332\001\006\000\003\000\000\000\001\000\002\000\000\001\017\000\002\000\000\000\006\000\000\000\340\001\020\000\002\000\000\000\n\000\000\000\346\001\022\000\003\000\000\000\001\000\001\000\000\001\025\000\003\000\000\000\001\000\003\000\000\001\032\000\005\000\000\000\001\000\000\000\360\001\033\000\005\000\000\000\001\000\000\000\370\001(\000\003\000\000\000\001\000\002\000\000\0011\000\002\000\000\000\037\000\000\001\000\0012\000\002\000\000\000\024\000\000\001\037\001<\000\002\000\000\000\n\000\000\0013\002\023\000\003\000\000\000\001\000\001\000\000\207i\000\004\000\000\000\001\000\000\001@\210%\000\004\000\000\000\001\000\000\003\274\000\000\004\320\000\010\000\010\000\010Appl

DOWNLOAD MODEL

In [8]:
#DOWNLOAD MODEL
import tarfile
import urllib.request

# Download and extract model
MODEL_DATE = '20200711'
MODEL_NAME = PRETRAINED_MODEL_NAME
MODEL_TAR_FILENAME = MODEL_NAME + '.tar.gz'
MODELS_DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/'
MODEL_DOWNLOAD_LINK = MODELS_DOWNLOAD_BASE + MODEL_DATE + '/' + MODEL_TAR_FILENAME
PATH_TO_MODEL_TAR = os.path.join(paths['PRETRAINED_MODEL_PATH'], MODEL_TAR_FILENAME)
PATH_TO_CKPT = os.path.join(paths['PRETRAINED_MODEL_PATH'], os.path.join(MODEL_NAME, 'checkpoint/'))
PATH_TO_CFG = os.path.join(paths['PRETRAINED_MODEL_PATH'], os.path.join(MODEL_NAME, 'pipeline.config'))
if not os.path.exists(PATH_TO_CKPT):
    print('Downloading model. This may take a while... ', end='')
    urllib.request.urlretrieve(MODEL_DOWNLOAD_LINK, PATH_TO_MODEL_TAR)
    tar_file = tarfile.open(PATH_TO_MODEL_TAR)
    tar_file.extractall(paths['PRETRAINED_MODEL_PATH'])
    tar_file.close()
    os.remove(PATH_TO_MODEL_TAR)
    print('Done')

# Download labels file
#LABEL_FILENAME = 'mscoco_label_map.pbtxt'
#LABELS_DOWNLOAD_BASE = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
#PATH_TO_LABELS = os.path.join(paths['PRETRAINED_MODEL_PATH'], os.path.join(MODEL_NAME, LABEL_FILENAME))
#if not os.path.exists(PATH_TO_LABELS):
    #print('Downloading label file... ', end='')
    #urllib.request.urlretrieve(LABELS_DOWNLOAD_BASE + LABEL_FILENAME, PATH_TO_LABELS)
    #print('Done')

In [10]:
#copy config file
!copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

        1 file(s) copied.


In [11]:
#update config for transfer learning
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
#change config parameters
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)
    

#checkpointpath = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
#newPath = dpath.replace(os.sep, '/')
#print(newPath)
       
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
#pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-4')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)
    
print(config_text)

model {
  ssd {
    num_classes: 2
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 4e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.01
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.997
          scale: true
          epsilon: 0.001
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      arg

TRAINING SCRIPT

In [12]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'model_main_tf2.py')

In [13]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=60000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [14]:
print(command)

python object_detection\model_main_tf2.py --model_dir=workspace_tiffanies\my_ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\model_data\model --pipeline_config_path=workspace_tiffanies\my_ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\model_data\model\pipeline.config --num_train_steps=60000


EVAL SCRIPT

In [15]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [16]:
print(command)

python object_detection\model_main_tf2.py --model_dir=workspace_tiffanies\my_ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\model_data\model --pipeline_config_path=workspace_tiffanies\my_ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\model_data\model\pipeline.config --checkpoint_dir=workspace_tiffanies\my_ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\model_data\model
